In [1]:
import  pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re 
import pyautogui
import warnings
warnings.filterwarnings("ignore")

# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비
columns = ["검색어", "순번", "한자","품사","설명","링크"]
df = pd.DataFrame(columns=columns)

excutable_path = "chromedriver.exe"

driver = webdriver.Chrome(executable_path=excutable_path)

base_url = 'https://stdict.korean.go.kr'

def urlmake(links):
    source = links
    driver.get(source)
    req = driver.page_source
    soup = BeautifulSoup(req, "html.parser") 
    return soup

def listmake(word):
    num = isNone(word.select_one('.t_blue1 > sup'))
    china = isNone(word.select_one('dt> span'))
    poumsa =  isNone(word.select_one('dt'))
    content = isNone(word.select_one('font'))
    link = word.select_one('a').attrs['href']
    return num,china,poumsa,content,base_url +link

def isNone(word):
    if word != None and word.text != '':
        if len(word) > 3:
            index = startAndLast(re.sub('[\t\n\s]','',word.text))
            if index == None:
                return "NOPE"
            else :
                return re.sub('[\t\n\s]','',word.text)[index[0]:index[1]]
        return re.sub('[\t\n\s]','',word.text)
    else:
        return "NOPE"

def startAndLast(word):
    start_index = 0
    last_index = 4
    for i in range(len(word)):
        if word[i] == '「':
            start_index = i
        if word[i] == '」':
            last_index = i+1         
            return start_index,last_index

        
word_lists = []
with open('word.txt', 'r',encoding = 'utf8') as file: 
    
    x = file.readlines()
    for i in range(len(x)):
        
        url = 'https://stdict.korean.go.kr/search/searchResult.do?pageSize=10&searchKeyword=' + x[i]

        search_soup = urlmake(url)

        title = search_soup.select_one('.t_blue2')
        if title != None:
            title = title.text.strip()
            words = search_soup.select('.result > li')
            for word in words:
                print(f'{columns[0]} : {title}')
                word_list = {}
                for i in range(len(listmake(word))):
                    print(f'{columns[i+1]} : {listmake(word)[i]}') 
                    word_list.setdefault(columns[i+1] , listmake(word)[i])
                word_lists.append(word_list)
                print('----------------------------------\n')
                row = [title,*listmake(word)]
                series = pd.Series(row, index=df.columns)
                df = df.append(series, ignore_index=True)
        else:
            text = search_soup.select_one('.no_result>li>div')
            if text == None:
                print(search_soup.select_one('.no_result').text)
            else:
                print(f'{columns[0]} : {x[i].strip()}')
                print(text.text.strip())
                print('https://opendic.korean.go.kr/search/searchResult?query='+x[i])

검색어 : 학원
순번 : 1
한자 : (學員)
품사 : 「명사」
설명 : 공부하는인원.
링크 : https://stdict.korean.go.kr/search/searchView.do?word_no=362856&searchKeywordTo=3
----------------------------------

검색어 : 학원
순번 : 2
한자 : (學院)
품사 : 「명사」
설명 : 『교육』일정한목적ㆍ교과과정ㆍ설비ㆍ제도및법규에의하여계속적으로학생에게교육을실시하는기관.=학교.
링크 : https://stdict.korean.go.kr/search/searchView.do?word_no=503671&searchKeywordTo=3
----------------------------------

검색어 : 학원
순번 : 3
한자 : (學園)
품사 : 「명사」
설명 : 『교육』학교및기타교육기관을통틀어이르는말.
링크 : https://stdict.korean.go.kr/search/searchView.do?word_no=362857&searchKeywordTo=3
----------------------------------

검색어 : 학원
순번 : 4
한자 : (學園)
품사 : 「명사」
설명 : 『문학』1919년에발간된최초의학생잡지.창가,소설따위가발표되어소년ㆍ소녀들에게처음으로문학적인분위기에접하는계기를만들어주었다.
링크 : https://stdict.korean.go.kr/search/searchView.do?word_no=362858&searchKeywordTo=3
----------------------------------

검색어 : 학교
순번 : NOPE
한자 : (學校)
품사 : 「명사」
설명 : 『교육』일정한목적ㆍ교과과정ㆍ설비ㆍ제도및법규에의하여계속적으로학생에게교육을실시하는기관.≒학원.
링크 : https://stdict.korean.go.kr/search/searchView.do?word_no=361333&searchKeywordTo=3
-------------

In [2]:
word_lists

[{'순번': '1',
  '한자': '(學員)',
  '품사': '「명사」',
  '설명': '공부하는인원.',
  '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=362856&searchKeywordTo=3'},
 {'순번': '2',
  '한자': '(學院)',
  '품사': '「명사」',
  '설명': '『교육』일정한목적ㆍ교과과정ㆍ설비ㆍ제도및법규에의하여계속적으로학생에게교육을실시하는기관.=학교.',
  '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=503671&searchKeywordTo=3'},
 {'순번': '3',
  '한자': '(學園)',
  '품사': '「명사」',
  '설명': '『교육』학교및기타교육기관을통틀어이르는말.',
  '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=362857&searchKeywordTo=3'},
 {'순번': '4',
  '한자': '(學園)',
  '품사': '「명사」',
  '설명': '『문학』1919년에발간된최초의학생잡지.창가,소설따위가발표되어소년ㆍ소녀들에게처음으로문학적인분위기에접하는계기를만들어주었다.',
  '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=362858&searchKeywordTo=3'},
 {'순번': 'NOPE',
  '한자': '(學校)',
  '품사': '「명사」',
  '설명': '『교육』일정한목적ㆍ교과과정ㆍ설비ㆍ제도및법규에의하여계속적으로학생에게교육을실시하는기관.≒학원.',
  '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=361333&searchKeywordTo=3'},
 {'순번': '1',
  '한자': '(力士)',
  '품사': '「명사」',
  

In [3]:
df

,검색어,순번,한자,품사,설명,링크
0,학원,1,(學員),「명사」,공부하는인원.,https://stdict.korean.go.kr/search/searchView....
1,학원,2,(學院),「명사」,『교육』일정한목적ㆍ교과과정ㆍ설비ㆍ제도및법규에의하여계속적으로학생에게교육을실시하는기관....,https://stdict.korean.go.kr/search/searchView....
2,학원,3,(學園),「명사」,『교육』학교및기타교육기관을통틀어이르는말.,https://stdict.korean.go.kr/search/searchView....
3,학원,4,(學園),「명사」,"『문학』1919년에발간된최초의학생잡지.창가,소설따위가발표되어소년ㆍ소녀들에게처음으로문...",https://stdict.korean.go.kr/search/searchView....
4,학교,NOPE,(學校),「명사」,『교육』일정한목적ㆍ교과과정ㆍ설비ㆍ제도및법규에의하여계속적으로학생에게교육을실시하는기관....,https://stdict.korean.go.kr/search/searchView....
5,역사,1,(力士),「명사」,뛰어나게힘이센사람.,https://stdict.korean.go.kr/search/searchView....
6,역사,2,(役事),「명사」,『건설』토목이나건축따위의공사.,https://stdict.korean.go.kr/search/searchView....
7,역사,3,(歷史),「명사」,"인류사회의변천과흥망의과정.또는그기록.≒사,춘추.",https://stdict.korean.go.kr/search/searchView....
8,역사,4,(歷事/歷仕),「명사」,여러대의임금을내리섬김.,https://stdict.korean.go.kr/search/searchView....
9,역사,5,(歷辭),「명사」,"『역사』조선시대에,새로임명된벼슬아치가감독관아를돌아다니며인사하던일.그벼슬에임명된지10...",https://stdict.korean.go.kr/search/searchView....


In [4]:
df.to_csv('./result.csv',encoding = 'utf-8')

In [5]:
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client['testitem-database']
collection = db['test_collection']

for item_list in word_lists:
    collection.insert_one(item_list)

In [6]:
item_list = collection.find()

for item in item_list:
    print(item)

{'_id': ObjectId('61fe0b12808f5951b4dbedba'), '순번': '1', '한자': '(學員)', '품사': '「명사」', '설명': '공부하는인원.', '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=362856&searchKeywordTo=3'}
{'_id': ObjectId('61fe0b12808f5951b4dbedbb'), '순번': '2', '한자': '(學院)', '품사': '「명사」', '설명': '『교육』일정한목적ㆍ교과과정ㆍ설비ㆍ제도및법규에의하여계속적으로학생에게교육을실시하는기관.=학교.', '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=503671&searchKeywordTo=3'}
{'_id': ObjectId('61fe0b12808f5951b4dbedbc'), '순번': '3', '한자': '(學園)', '품사': '「명사」', '설명': '『교육』학교및기타교육기관을통틀어이르는말.', '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=362857&searchKeywordTo=3'}
{'_id': ObjectId('61fe0b12808f5951b4dbedbd'), '순번': '4', '한자': '(學園)', '품사': '「명사」', '설명': '『문학』1919년에발간된최초의학생잡지.창가,소설따위가발표되어소년ㆍ소녀들에게처음으로문학적인분위기에접하는계기를만들어주었다.', '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=362858&searchKeywordTo=3'}
{'_id': ObjectId('61fe0b12808f5951b4dbedbe'), '순번': 'NOPE', '한자': '(學校)', '품사': '「명사」', '설명': '『교육』일정한목적ㆍ교과과정ㆍ설

In [7]:
# collection.drop() // 콜렉션 자체를 삭제
collection.delete_many({}) # 조건을 {}을 주실 콜렉션 안의 데이터들을 다 삭제

In [8]:
collection.insert_many(word_lists)

In [9]:
item_list = collection.find()

for item in item_list:
    print(item)

{'_id': ObjectId('61fe0e04a8ee4b8574d98ed3'), '순번': '1', '한자': '(學員)', '품사': '「명사」', '설명': '공부하는인원.', '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=362856&searchKeywordTo=3'}
{'_id': ObjectId('61fe0e04a8ee4b8574d98ed4'), '순번': '2', '한자': '(學院)', '품사': '「명사」', '설명': '『교육』일정한목적ㆍ교과과정ㆍ설비ㆍ제도및법규에의하여계속적으로학생에게교육을실시하는기관.=학교.', '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=503671&searchKeywordTo=3'}
{'_id': ObjectId('61fe0e04a8ee4b8574d98ed5'), '순번': '3', '한자': '(學園)', '품사': '「명사」', '설명': '『교육』학교및기타교육기관을통틀어이르는말.', '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=362857&searchKeywordTo=3'}
{'_id': ObjectId('61fe0e04a8ee4b8574d98ed6'), '순번': '4', '한자': '(學園)', '품사': '「명사」', '설명': '『문학』1919년에발간된최초의학생잡지.창가,소설따위가발표되어소년ㆍ소녀들에게처음으로문학적인분위기에접하는계기를만들어주었다.', '링크': 'https://stdict.korean.go.kr/search/searchView.do?word_no=362858&searchKeywordTo=3'}
{'_id': ObjectId('61fe0e04a8ee4b8574d98ed7'), '순번': 'NOPE', '한자': '(學校)', '품사': '「명사」', '설명': '『교육』일정한목적ㆍ교과과정ㆍ설